In [1]:
import sys
import os
import enum 
import asyncio
###
from datetime import datetime
from math import floor, ceil
import time
from time import sleep
###
import json
###
from dotenv import load_dotenv
load_dotenv()
###
import zmq
zContext = zmq.asyncio.Context()
###
import logging
logger = logging.getLogger()
# from zmq.log.handlers import PUBHandler
# zmq_log_handler = PUBHandler('tcp://127.0.0.1:55356')
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} > {message}', style='{'))
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} #{lineno:>3} > {message}', style='{'), logging.DEBUG)
# zmq_log_handler.setRootTopic('greeter')
# logger.addHandler(zmq_log_handler)

logger.setLevel(logging.DEBUG)
###
from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
client = MongoClient()
db=client.fsmbot
###
import ccxt
exchange = getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})
###
from trade_machine import TradeModel, trades, load_trades

from dotmap import DotMap

price_ticker = {}

import zmq
import zmq.asyncio
zContext = zmq.asyncio.Context()

In [2]:
async def run_ticker():
    socket = zContext.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        _, symbol, ticker = tickerString.split()
        ticker = json.loads(ticker)
        price_ticker[symbol] = ticker
        if symbol in trades:
            for model in trades[symbol].values():
                if model.state in ['LIVE', 'LONG']:
                    model.tick(ticker=ticker)

from threading import Thread

ticker_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

ticker_thread = Thread(target=setup_loop, args=(ticker_loop,))
ticker_thread.start()
ticker_future = asyncio.run_coroutine_threadsafe(run_ticker(), ticker_loop)  

In [3]:
logger.setLevel(logging.INFO)
exchange.load_markets()
trades = load_trades(exchange, db.trades)
trades

INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'


{'ADAUSDT': {'618085b7dacef867e5df6e7f': 618085b7dacef867e5df6e7f LONG ADA/USDT 49.9}}

load_trades_from_tb()

In [4]:
def normalize_values(symbol, qty, trigger, stop, stop_limit, target):
    return {'qty': exchange.amount_to_precision(symbol, qty), 
        'trigger': exchange.price_to_precision(symbol, trigger),
        'stop': exchange.price_to_precision(symbol, stop),
        'stop_limit': exchange.price_to_precision(symbol, stop_limit),
        'target': exchange.price_to_precision(symbol, target)
    }


def calc_trade(capital, risk_perc, symbol, trigger, stop, target):
    market = exchange.market(symbol)
    ntrigger = float(exchange.price_to_precision(symbol, trigger))
    nstop = float(exchange.price_to_precision(symbol, stop))
    nstop_limit = float(exchange.price_to_precision(symbol, stop * 0.95))
    ntarget = float(exchange.price_to_precision(symbol, target))
    risk_size = capital * risk_perc
    nrisk_size = float(exchange.cost_to_precision(symbol, risk_size))

    qty = float(exchange.amount_to_precision(symbol, risk_size / (trigger - stop)))
    size = qty * trigger
    nsize = float(exchange.cost_to_precision(symbol, size))
    profit_perc = round((target - trigger) / trigger * 100, 2)
    loss_perc = round((trigger - stop) / trigger * 100, 2)
    r2r = round(profit_perc / loss_perc, 2)
    return {'symbol': symbol, 'market': market['id'], 'size': size, 'risk_size': nrisk_size, 'qty': qty, 'r2r': r2r, 'profit_perc' : profit_perc, 'loss_perc' : loss_perc, 
        'trigger': ntrigger, 'stop': nstop, 'stop_limit': nstop_limit, 'target': ntarget}


def create_trade(capital, risk_perc, symbol, trigger, stop, target):
    trade_values = calc_trade(capital, risk_perc, symbol, trigger, stop, target)
    trade = TradeModel(exchange, db.trades)
    # (nqty, ntrigger, nstop, nstop_limit, ntarget) = normalize_values(symbol, qty, trigger, stop, stop_limit, target)
    trade.insert(data={"state": 'DISABLED', "symbol" : symbol, **trade_values})
    trade.start()
    trade.watch()
    return trade


In [5]:
symbol = 'ADA/USDT'
market = exchange.market(symbol)
nsymbol = market['id']
amount = 20.0

exchange.fetchTicker(symbol)
ticker = exchange.fetchTicker(symbol)
last_price = ticker['last']
# trade = create_trade(200, 0.01, symbol, last_price + 0.002, last_price - 0.002, last_price + 0.002)
# trade
trade = create_trade(200, 0.01, symbol, last_price + 0.02, last_price - 0.02, last_price + 0.02)

INFO:transitions.core:Executed callback 'insertrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Finished processing state DISABLED exit callbacks.
INFO:transitions.core:Finished processing state LIVE enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'buy_on_exchange'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'set_oco_on_exchange'
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'set_oco_on_exchange'
INFO:transitions.core:Finished processing state LONG exit callbacks.
INFO:transitions.core:Finished processing state LONG_OCO enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


In [5]:
trade = trades['ADAUSDT']['618085b7dacef867e5df6e7f']
# trade.tick({'eventType': '24hrTicker', 'eventTime': 1635812425874, 'symbol': 'ADAUSDT', 'priceChange': '-0.02400000', 'priceChangePercent': '-1.217', 'weightedAvg': '1.96039439', 'prevDayClose': '1.97200000', 'curDayClose': '1.94800000', 'closeTradeQuantity': '113.00000000', 'bestBid': '1.94800000', 'bestBidQnt': '19309.10000000', 'bestAsk': '1.94900000', 'bestAskQnt': '49035.80000000', 'open': '1.97200000', 'high': '2.01900000', 'low': '1.91100000', 'volume': '142367482.50000000', 'volumeQuote': '279096414.06090000', 'openTime': 1635726025874, 'closeTime': 1635812425874, 'firstTradeId': 307530389, 'lastTradeId': 307972281, 'totalTrades': 441893})
# trade = trades[nsymbol]
# trade = create
# trade.removeoco(data={'$set': {'stop': 1.940}})
trade.canceloco()
# trade.setoco()
# trade.removestop()

INFO:transitions.core:Executed callback 'set_oco_on_exchange'
INFO:transitions.core:Finished processing state LONG exit callbacks.
INFO:transitions.core:Finished processing state LONG_OCO enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


True

In [ ]:
# # trades
# for symbol, trade_chunk_arr in trades.items():
#     for chunk in trade_chunk_arr:
#         model = chunk['model'] 
#         machine = chunk['machine']
#         if model.state != 'LONG':
#             continue
#         trade = model.trade
#         stop_order = exchange.fetchOrder(trade['curStopResp']['id'], symbol=symbol)
#         if exchange.fetchOrderStatus(trade['curStopResp']['id'], symbol='ALGOUSDT') == 'closed':
#             print(trade['id'], 'closed')
#         # print(chunk)
#     # print(tradeChunk)
# # tradeModel.trade

In [9]:
# logger.setLevel(logging.INFO)
# trade.state, trade.rec.symbol
trades = load_trades(exchange, db.trades)
trades

INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'


{'ADA/USDT': {'618073ce79057cc85dca385e': 618073ce79057cc85dca385e LIVE ADA/USDT 499.9}}

INFO:transitions.core:Executed callback 'insertrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Finished processing state DISABLED exit callbacks.
INFO:transitions.core:Finished processing state LIVE enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


In [15]:
trade = trades['ADAUSDT']['6180601e017cac0f83038874']
trade.delete()

KeyError: 'ADAUSDT'

In [26]:
trades

{'ADAUSDT': {'6180601e017cac0f83038874': 6180601e017cac0f83038874 LONG ADAUSDT 49.99999999999996}}